In [ ]:
import geopandas as gpd
import panel as pn
import yaml
import numpy as np
import base64
import math
import time
import xyzservices.providers as xyz
import asyncio
import yaml
from random import sample
from bokeh.palettes import Category10_10 as palette
from bokeh.io import curdoc
from bokeh.models import (
    ColumnDataSource, 
    PointDrawTool, 
    Button,
    HoverTool,
    Range1d
)
from bokeh.plotting import Column, figure, show
from shapely import Point
from copy import deepcopy

#SET PANEL EXTENSION
pn.extension(notifications=True)
pn.state.notifications.position = "center-center"

#READ & lOAD YAML PARAMETERS
PARAMETERS = "game_params.yml"
with open(PARAMETERS, "r") as file:
    params = yaml.load(
        file,
        Loader=yaml.FullLoader,
    )
RANGES = params["ranges"]
BUFFER_VALUE = params["buffer_value"]
NB_Q = params["nb_questions"]
CSV_SEP = params["csv"]["sep"]
CSV_FILE = params["csv"]["file"]
WIKI_URL = params["url"]


class Game:
    """
    Create and manage shortest paths
    visualisation figures
    """
    def __init__(self):
        """
        Init figures and widgets
        """
        self.rules = pn.pane.Alert(
            "<p>Click on the START button \
            to start a game.</p><p>Place a point on the map \
            (<i>with Point Draw Tool</i> \
            <img src='https://docs.bokeh.org/en/latest/_images/PointDraw.png' \
            alt='Point draw tool' style='width:20px;height:20px'>) \
            to answer a question (<i>out of a total of {}</i>)</p><p>Then click \
            on the VALIDATE button.</p><p>You can also pan \
            <img src='https://docs.bokeh.org/en/latest/_images/Pan.png' \
            alt='Pan tool' style='width:20px;height:20px'> \
            and wheel zoom \
            <img src='https://docs.bokeh.org/en/latest/_images/WheelZoom.png' \
            alt='Wheel zoom tool' style='width:20px;height:20px'></p>".format(NB_Q),
            alert_type="primary"
        )
        self.button_click_event = None
        self.start_time = time.time()
        self.progress_value = int(100/NB_Q)
        self.chat_feed = pn.chat.ChatFeed()
        self.map = figure(
            name="map",
            sizing_mode="scale_both",
            min_height=600
        )
        self.map.add_tile(xyz.OpenStreetMap.Mapnik)
        self.map.axis.visible = False
        self.map.grid.visible = False
        self.map.x_range = Range1d(
            RANGES["x"][0],
            RANGES["x"][1]
        )
        self.map.y_range = Range1d(
            RANGES["y"][0],
            RANGES["y"][1]
        )
        df = gpd.pd.read_csv(
            CSV_FILE,
            sep=CSV_SEP,
            encoding="utf-8"
        )
        df["geometry"] = df.apply(
            lambda x: Point(x.x, x.y),
            axis=1
        )
        self.gdf = gpd.GeoDataFrame(df).set_crs(
            epsg=4326
        ).to_crs(
            epsg=3857
        )
        self._set()
        self._get_random()
        self.progress = pn.indicators.Progress(
            name="Progress", 
            value=0, 
            width=200
        )
        self.points = self.map.scatter(
            x="x", 
            y="y", 
            source=self.source, 
            size=10
        )
        self.points.on_change(
            "data_source", 
            self._get_point
        )
        draw_tool = PointDrawTool(
            renderers=[self.points], 
            empty_value="black"
        )
        self.map.add_tools(draw_tool)
        self.map.toolbar.active_tap = draw_tool
        self.run_button = Button(
            label="START", 
            button_type="success"
        )
        self.reset_button = Button(
            label="RESET", 
            button_type="warning"
        )
        self.check_button = Button(
            label="VALIDATE", 
            button_type="success"
        )
        self.run_button.on_click(
            self._get_question
        )
        self.check_button.on_click(
            self._check
        )
        self.reset_button.on_click(
            self._reset
        )
        self.loading = pn.indicators.LoadingSpinner(
            value=True, 
            size=60, 
            name="spinner", 
            visible=False
        )
        tooltips = [
            ("question", "@question"),
            ("time", "@time")
        ]
        self.hist = figure(
            x_axis_label="Question",
            y_axis_label="Time in seconds",
            title="Time by question",
            toolbar_location=None, 
            tools="",
            x_range=[
                str(x+1) for x in range(NB_Q)
            ],
            tooltips=tooltips
        )
        self.hist.vbar(
            x="question",
            top="time",
            width=0.5,
            bottom=0.0,
            source=self.hist_source
        )

    
    def _set(self, reset=False):
        """

        """
        self.question_time = {
            "question":[],
            "time":[]
        }
        source = {
                "x": [], 
                "y": [] 
        }
        hist_source = {
            "question":[],
            "time":[]
        }
        if reset is True:
            self.source.data = source
            self.hist_source.data = hist_source
        else:
            self.source = ColumnDataSource(source)
            self.hist_source = ColumnDataSource(hist_source)
        self.index = 0
    
    
    def _get_question(self, event):
        """

        """
        if self.index == NB_Q:
            self.chat_feed.send(
                {
                    "object":"Game over. Click on \
                    the RESET button to start a new game.\
                    Total time: {}".format(
                        time.strftime(
                            "%Hh%Mm%Ss", 
                            time.gmtime(
                                time.time() - self.start_time
                            )
                        )
                    ),
                    "user":"Bot",
                    "avatar":"🤖"
                },
                respond=False
            )
            time.sleep(0.5)
            self.chat_feed.send(
                {
                    "object":self.hist,
                    "user":"Bot",
                    "avatar":"🤖"
                },
                respond=False
            )
            self.check_button.disabled
        else:
            if self.button_click_event is None:
                self.button_click_event = event
            self.question_time_start = time.time()
            self.selection = self.game_set.iloc[
                self.index
            ]
            self.chat_feed.send(
                {
                    "object":"Question {}/{}: {}".format(
                        self.index+1,
                        NB_Q,
                        self.selection.question
                    ),
                    "user":"Bot",
                    "avatar":"🤖"
                },
                respond=False
            ) 
            self.index += 1
            self.run_button.disabled = True
        

    def _get_random(self):
        """

        """
        self.game_set = self.gdf.copy().iloc[
            sample(
                list(self.gdf.index), 
                k=NB_Q
            )
        ]
        self.indexes = list(
            set(
                self.game_set.index.values
            )
        )
        self.game_set.reset_index(
            drop=True,
            inplace=True
        )
        
    
    def _get_point(self, attr, old, new):
        """
        Get the starting and ending coordinates
        points (points added by user)
        """
        self.answer_point = (
            self.source.data["x"].values[0],
            self.source.data["y"].values[0]
        )
    
    
    def _reset(self, event):
        """
        Reset data points and multilines 
        """
        self.run_button.disabled = False
        self.chat_feed.clear()
        self.progress.value = 0
        self.button_click_event = None
        self._set(reset=True)

    
    def _check(self, event):
        """

        """
        column.loading = True
        gdf = gpd.GeoDataFrame(
            self.source.data
        )
        if gdf.empty is True:
            column.loading = False
            pn.state.notifications.error(
                "Please place a point", 
                duration=2000
            )
        else:
            gdf["geometry"] = gdf.apply(
                lambda x: Point(x.x, x.y),
                axis=1
            )
            gdf = gpd.GeoDataFrame(
                gdf
            ).set_geometry(
                "geometry"
            ).set_crs(
                epsg=3857
            )
            buffer = gdf.geometry.values[-1].buffer(
                BUFFER_VALUE
            )
            if buffer.contains(
                self.selection.geometry
            ) is True:
                self.progress.value += self.progress_value
                self.question_time["question"].append(str(self.index))
                time_question = time.time()-self.question_time_start
                self.question_time["time"].append(
                    time_question
                )
                self.hist_source.data = self.question_time
                column.loading = False
                self.run_button.disabled = False
                chat_url = "{}{}".format(
                    WIKI_URL,
                    self.selection.answer.replace(" ","_")
                )
                time.sleep(1)
                self.chat_feed.send(
                    {
                        "object":"Great !\
                        You have found <a href='{}' target='_blank'>{}</a>\
                        in {} seconds".format(
                            chat_url,
                            self.selection.answer,
                            time_question
                        ),
                        "user":"Bot", 
                        "avatar":"🤖"
                    },
                    respond=False
                )
                pn.state.notifications.success(
                    "Great", 
                    duration=1000
                )
                time.sleep(0.5)
                self._get_question(self.button_click_event)
            else:
                column.loading = False
                pn.state.notifications.error(
                    "Nope ! Try again !", 
                    duration=2000
                )
                self.chat_feed.send(
                    {
                        "object":"Nope ! Here's a hint: {}".format(
                            self.selection.hint
                        ),
                        "user":"Bot", 
                        "avatar":"🤖"
                    },
                    respond=False
                )
        
game = Game()
pn.extension(
    loading_spinner="dots",
    loading_color="#00aa41",
    sizing_mode="stretch_both",
)
# Template
bootstrap = pn.template.BootstrapTemplate(
    title="Doctoriales GAME",
    # theme=pn.template.theme.DarkTheme,
)
column = pn.Row(
    game.map,
    game.chat_feed,
    sizing_mode="scale_both",
    min_height=5000
)
bootstrap.sidebar_width = 250
bootstrap.sidebar.append(
    pn.Row(
        game.run_button,
        game.check_button,
        game.reset_button
    )
)
bootstrap.sidebar.append(
    pn.Column(
        game.progress
    )
)
bootstrap.sidebar.append(game.rules)
bootstrap.main.append(column)
bootstrap.servable();